# Implement and train a LSTM for sentiment analysis

## Step 0: set up the environment

In [1]:
import functools
import sys
import numpy as np
import pandas as pd
import random
import re
import matplotlib.pyplot as plt
import tqdm
import nltk
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from collections import Counter
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset

nltk.download('stopwords')

torch.backends.cudnn.benchmark = True

import os
os.makedirs("resources", exist_ok=True)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


### Hyperparameters. No need to touch.

In [2]:
class HyperParams:
    def __init__(self):
        # Constance hyperparameters. They have been tested and don't need to be tuned.
        self.PAD_INDEX = 0
        self.UNK_INDEX = 1
        self.PAD_TOKEN = '<pad>'
        self.UNK_TOKEN = '<unk>'
        self.STOP_WORDS = set(stopwords.words('english'))
        self.MAX_LENGTH = 256
        self.BATCH_SIZE = 96
        self.EMBEDDING_DIM = 1
        self.HIDDEN_DIM = 100
        self.OUTPUT_DIM = 2
        self.N_LAYERS = 1
        self.DROPOUT_RATE = 0.0
        self.LR = 0.001
        self.N_EPOCHS = 5
        self.WD = 0
        self.SEED = 12
        self.BIDIRECTIONAL = False

## Lab 1(a) Implement your own data loader function.  
First, you need to read the data from the dataset file on the local disk.
Then, split the dataset into three sets: train, validation and test by 7:1:2 ratio.
Finally return x_train, x_valid, x_test, y_train, y_valid, y_test where x represents reviews and y represent labels.  

In [4]:
def load_imdb(base_csv:str = './IMDBDataset.csv'):
    """
    Load the IMDB dataset
    :param base_csv: the path of the dataset file.
    :return: train, validation and test set.
    """
    # Add your code here.
def load_imdb(base_csv:str = './IMDBDataset.csv'):
    """
    Load the IMDB dataset
    :param base_csv: the path of the dataset file.
    :return: train, validation and test set.
    """
    # Add your code here.
    df = pd.read_csv(base_csv)

    x, y = df["review"].values, df["sentiment"].values

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=100)

    x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.125, random_state=100)

    print(f'shape of train data is {x_train.shape}')
    print(f'shape of test data is {x_test.shape}')
    print(f'shape of valid data is {x_valid.shape}')
    return x_train, x_valid, x_test, y_train, y_valid, y_test


## Lab 1(b): Implement your function to build a vocabulary based on the training corpus.
Implement the build_vocab function to build a vocabulary based on the training corpus.
You should first compute the frequency of all the words in the training corpus. Remove the words
that are in the STOP_WORDS. Then filter the words by their frequency (≥ min_freq) and finally
generate a corpus variable that contains a list of words.

In [5]:
def build_vocab(x_train:list, min_freq: int=5, hparams=None) -> dict:
    """
    build a vocabulary based on the training corpus.
    :param x_train:  List. The training corpus. Each sample in the list is a string of text.
    :param min_freq: Int. The frequency threshold for selecting words.
    :return: dictionary {word:index}
    """
    # Add your code here. Your code should assign corpus with a list of words.
    all_words = ' '.join(x_train).split()
    all_words = [word.lower() for word in all_words]
    all_words = [re.sub(r'[^\w\s]', '', word) for word in all_words]

    # Remove stop words
    all_words_filtered = [word for word in all_words if word not in hparams.STOP_WORDS]

    corpus = dict(Counter(all_words_filtered))

    corpus_ = [word for word, freq in corpus.items() if freq >= min_freq]

    # Create vocabulary dictionary with indexing
    vocab = {word: i + 2 for i, word in enumerate(corpus_)}
    vocab[hparams.PAD_TOKEN] = hparams.PAD_INDEX
    vocab[hparams.UNK_TOKEN] = hparams.UNK_INDEX

    return vocab


## Lab 1(c): Implement your tokenize function.
For each word, find its index in the vocabulary.
Return a list of int that represents the indices of words in the example.

In [6]:
def tokenize(vocab: dict, example: str)-> list:
    """
    Tokenize the give example string into a list of token indices.
    :param vocab: dict, the vocabulary.
    :param example: a string of text.
    :return: a list of token indices.
    """
    # Your code here.
    tokens = example.split()
    tokens = [re.sub(r'[^\w\s]', '', token) for token in tokens]
    tokens = [token.lower() for token in tokens]

    # Convert tokens to indices using the vocabulary
    indices = [vocab.get(token, vocab.get('<unk>')) for token in tokens]

    return indices

## Lab 1 (d): Implement the __getitem__ function. Given an index i, you should return the i-th review and label.
The review is originally a string. Please tokenize it into a sequence of token indices.
Use the max_length parameter to truncate the sequence so that it contains at most max_length tokens.
Convert the label string ('positive'/'negative') to a binary index. 'positive' is 1 and 'negative' is 0.
Return a dictionary containing three keys: 'ids', 'length', 'label' which represent the list of token ids, the length of the sequence, the binary label.

In [7]:
class IMDB(Dataset):
    def __init__(self, x, y, vocab, max_length=256) -> None:
        """
        :param x: list of reviews
        :param y: list of labels
        :param vocab: vocabulary dictionary {word:index}.
        :param max_length: the maximum sequence length.
        """
        self.x = x
        self.y = y
        self.vocab = vocab
        self.max_length = max_length

    def __getitem__(self, idx: int):
        """
        Return the tokenized review and label by the given index.
        :param idx: index of the sample.
        :return: a dictionary containing three keys: 'ids', 'length', 'label' which represent the list of token ids, the length of the sequence, the binary label.
        """
        # Add your code here.
        label = self.y[idx]
        label = 1 if label == 'positive' else 0

        review = self.x[idx]
        ids = tokenize(self.vocab, review)
        if len(ids) > self.max_length:
            ids = ids[:self.max_length]
        length = len(ids)

        return {'ids': ids, 'length': length, 'label': label}


    def __len__(self) -> int:
        return len(self.x)

def collate(batch, pad_index):
    batch_ids = [torch.LongTensor(i['ids']) for i in batch]
    batch_ids = nn.utils.rnn.pad_sequence(batch_ids, padding_value=pad_index, batch_first=True)
    batch_length = torch.Tensor([i['length'] for i in batch])
    batch_label = torch.LongTensor([i['label'] for i in batch])
    batch = {'ids': batch_ids, 'length': batch_length, 'label': batch_label}
    return batch

collate_fn = collate

## Lab 1 (e): Implement the LSTM model for sentiment analysis.
Q(a): Implement the initialization function.
Your task is to create the model by stacking several necessary layers including an embedding layer, a lstm cell, a linear layer, and a dropout layer.
You can call functions from Pytorch's nn library. For example, nn.Embedding, nn.LSTM, nn.Linear.<br>
Q(b): Implement the forward function.
    Decide where to apply dropout.
    The sequences in the batch have different lengths. Write/call a function to pad the sequences into the same length.
    Apply a fully-connected (fc) layer to the output of the LSTM layer.
    Return the output features which is of size [batch size, output dim].

In [8]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

def init_weights(m):
    if isinstance(m, nn.Embedding):
        nn.init.xavier_normal_(m.weight)
    elif isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        nn.init.zeros_(m.bias)
    elif isinstance(m, nn.LSTM) or isinstance(m, nn.GRU):
        for name, param in m.named_parameters():
            if 'bias' in name:
                nn.init.zeros_(param)
            elif 'weight' in name:
                nn.init.orthogonal_(param)

class LSTM(nn.Module):
    def __init__(
        self,
        vocab_size: int,
        embedding_dim: int,
        hidden_dim: int,
        output_dim: int,
        n_layers: int,
        dropout_rate: float,
        pad_index: int,
        bidirectional: bool,
        **kwargs):
        """
        Create a LSTM model for classification.
        :param vocab_size: size of the vocabulary
        :param embedding_dim: dimension of embeddings
        :param hidden_dim: dimension of hidden features
        :param output_dim: dimension of the output layer which equals to the number of labels.
        :param n_layers: number of layers.
        :param dropout_rate: dropout rate.
        :param pad_index: index of the padding token.we
        """
        super().__init__()
        # Add your code here. Initializing each layer by the given arguments. (you can use nn.LSTM, nn.Embedding, nn.Linear, nn.Dropout)
        # Initialize embedding layer
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embedding_dim, padding_idx=pad_index)
        # Initialize LSTM layer
        self.lstm_layer = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim, num_layers=n_layers,
                                  dropout=dropout_rate, batch_first=True, bidirectional=bidirectional)
        # Initialize fully connected layer
        self.fc_layer = nn.Linear(in_features=hidden_dim, out_features=output_dim)
        # Initialize dropout layer
        self.dropout_layer = nn.Dropout(dropout_rate)
        # Weight initialization. DO NOT CHANGE!
        if "weight_init_fn" not in kwargs:
            self.apply(init_weights)
        else:
            self.apply(kwargs["weight_init_fn"])

    def forward(self, ids:torch.Tensor, length:torch.Tensor):
        """
        Feed the given token ids to the model.
        :param ids: [batch size, seq len] batch of token ids.
        :param length: [batch size] batch of length of the token ids.
        :return: prediction of size [batch size, output dim].
        """
        # Add your code here.
        # Embedding lookup
        embedded_tokens = self.embedding_layer(ids)
        embedded_tokens = self.dropout_layer(embedded_tokens)
        # Pack the sequences for LSTM processing
        packed_embedded = pack_padded_sequence(embedded_tokens, length, batch_first=True, enforce_sorted=False)
        packed_output, (hidden_state, cell_state) = self.lstm_layer(packed_embedded)
        # Unpack the sequences
        output, output_lengths = pad_packed_sequence(packed_output, batch_first=True)
        # Extract the last hidden state
        hidden_representation = self.dropout_layer(hidden_state[-1, :, :])
        # Compute the final prediction
        prediction = self.fc_layer(hidden_representation)

        return prediction

## Training Code (do not modify)

In [9]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


def train(dataloader, model, criterion, optimizer, scheduler, device):
    model.train()
    epoch_losses = []
    epoch_accs = []

    for batch in tqdm.tqdm(dataloader, desc='training...', file=sys.stdout):
        ids = batch['ids'].to(device)
        length = batch['length']
        label = batch['label'].to(device)
        prediction = model(ids, length)
        loss = criterion(prediction, label)
        accuracy = get_accuracy(prediction, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        epoch_losses.append(loss.item())
        epoch_accs.append(accuracy.item())
        scheduler.step()

    return epoch_losses, epoch_accs

def evaluate(dataloader, model, criterion, device):
    model.eval()
    epoch_losses = []
    epoch_accs = []

    with torch.no_grad():
        for batch in tqdm.tqdm(dataloader, desc='evaluating...', file=sys.stdout):
            ids = batch['ids'].to(device)
            length = batch['length']
            label = batch['label'].to(device)
            prediction = model(ids, length)
            loss = criterion(prediction, label)
            accuracy = get_accuracy(prediction, label)
            epoch_losses.append(loss.item())
            epoch_accs.append(accuracy.item())

    return epoch_losses, epoch_accs

def get_accuracy(prediction, label):
    batch_size, _ = prediction.shape
    predicted_classes = prediction.argmax(dim=-1)
    correct_predictions = predicted_classes.eq(label).sum()
    accuracy = correct_predictions / batch_size
    return accuracy

def predict_sentiment(text, model, vocab, device):
    tokens = tokenize(vocab, text)
    ids = [vocab[t] if t in vocab else UNK_INDEX for t in tokens]
    length = torch.LongTensor([len(ids)])
    tensor = torch.LongTensor(ids).unsqueeze(dim=0).to(device)
    prediction = model(tensor, length).squeeze(dim=0)
    probability = torch.softmax(prediction, dim=-1)
    predicted_class = prediction.argmax(dim=-1).item()
    predicted_probability = probability[predicted_class].item()
    return predicted_class, predicted_probability

### Learning rate warmup. DO NOT TOUCH!

In [10]:
class ConstantWithWarmup(torch.optim.lr_scheduler._LRScheduler):
    def __init__(
        self,
        optimizer,
        num_warmup_steps: int,
    ):
        self.num_warmup_steps = num_warmup_steps
        super().__init__(optimizer)

    def get_lr(self):
        if self._step_count <= self.num_warmup_steps:
            # warmup
            scale = 1.0 - (self.num_warmup_steps - self._step_count) / self.num_warmup_steps
            lr = [base_lr * scale for base_lr in self.base_lrs]
            self.last_lr = lr
        else:
            lr = self.base_lrs
        return lr

### Implement the training / validation iteration here.

In [11]:
import torch.nn.functional as F
import random

In [29]:
def train_and_test_model_with_hparams(hparams, model_type="lstm", **kwargs):
    # Seeding. DO NOT TOUCH! DO NOT TOUCH hparams.SEED!
    # Set the random seeds.
    torch.manual_seed(hparams.SEED)
    random.seed(hparams.SEED)
    np.random.seed(hparams.SEED)

    x_train, x_valid, x_test, y_train, y_valid, y_test = load_imdb()
    vocab = build_vocab(x_train, hparams=hparams)
    vocab_size = len(vocab)
    print(f'Length of vocabulary is {vocab_size}')

    train_data = IMDB(x_train, y_train, vocab, hparams.MAX_LENGTH)
    valid_data = IMDB(x_valid, y_valid, vocab, hparams.MAX_LENGTH)
    test_data = IMDB(x_test, y_test, vocab, hparams.MAX_LENGTH)

    collate = functools.partial(collate_fn, pad_index=hparams.PAD_INDEX)

    train_dataloader = torch.utils.data.DataLoader(
        train_data, batch_size=hparams.BATCH_SIZE, collate_fn=collate, shuffle=True)
    valid_dataloader = torch.utils.data.DataLoader(
        valid_data, batch_size=hparams.BATCH_SIZE, collate_fn=collate)
    test_dataloader = torch.utils.data.DataLoader(
        test_data, batch_size=hparams.BATCH_SIZE, collate_fn=collate)

    # Model

    model = LSTM(
            vocab_size,
            hparams.EMBEDDING_DIM,
            hparams.HIDDEN_DIM,
            hparams.OUTPUT_DIM,
            hparams.N_LAYERS,
            hparams.DROPOUT_RATE,
            hparams.PAD_INDEX,
            hparams.BIDIRECTIONAL,
            **kwargs)

    num_params = count_parameters(model)
    print(f'The model has {num_params:,} trainable parameters')


    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # DO NOT TOUCH optimizer-specific hyperparameters! (e.g., eps, momentum)
    # DO NOT change optimizer implementations!

    optimizer = optim.Adam(model.parameters(), lr=hparams.LR, weight_decay=hparams.WD, eps=1e-6)


    criterion = nn.CrossEntropyLoss()
    criterion = criterion.to(device)

    # Start training
    best_valid_loss = float('inf')

    # Warmup Scheduler. DO NOT TOUCH!
    WARMUP_STEPS = 200
    lr_scheduler = ConstantWithWarmup(optimizer, WARMUP_STEPS)

    for epoch in range(hparams.N_EPOCHS):

        # Your code: implement the training process and save the best model.
        # Train the model for one epoch
        train_loss, train_acc = train(
            dataloader=train_dataloader,
            model=model,
            criterion=criterion,
            optimizer=optimizer,
            scheduler=lr_scheduler,
            device=device
        )

        # Evaluate the model on the validation set
        valid_loss, valid_acc = evaluate(
            dataloader=valid_dataloader,
            model=model,
            criterion=criterion,
            device=device
        )

        epoch_train_loss = np.mean(train_loss)
        epoch_train_acc = np.mean(train_acc)
        epoch_valid_loss = np.mean(valid_loss)
        epoch_valid_acc = np.mean(valid_acc)

        # Save the model that achieves the smallest validation loss.
        if epoch_valid_loss < best_valid_loss:
            # Your code: save the best model somewhere (no need to submit it to Sakai)
            best_valid_loss = epoch_valid_loss
            model_save_path = f'best_{model_type}.pt'
            torch.save(model.state_dict(), model_save_path)


        print(f'epoch: {epoch+1}')
        print(f'train_loss: {epoch_train_loss:.3f}, train_acc: {epoch_train_acc:.3f}')
        print(f'valid_loss: {epoch_valid_loss:.3f}, valid_acc: {epoch_valid_acc:.3f}')


        # Your Code: Load the best model's weights.
        model_save_path = f'best_{model_type}.pt'
        model.load_state_dict(torch.load(model_save_path))


        # Your Code: evaluate test loss on testing dataset (NOT Validation)
        test_loss, test_acc = evaluate(
        dataloader=test_dataloader,
        model=model,
        criterion=criterion,
        device=device)


        epoch_test_loss = np.mean(test_loss)
        epoch_test_acc = np.mean(test_acc)
        print(f'test_loss: {epoch_test_loss:.3f}, test_acc: {epoch_test_acc:.3f}')

        # Your Code: select one of the entries in test set and predict its sentiment, print out the text, prediction and the probability.
        sample = test_data[0]

        sample_text_ids = sample['ids']
        sample_length = sample['length']
        sample_label = sample['label']

        sample_ids_tensor = torch.LongTensor(sample_text_ids).unsqueeze(0).to(device)
        sample_length_tensor = torch.tensor([sample_length], dtype=torch.int64).cpu()

        model.eval()
        with torch.no_grad():
             logits = model(sample_ids_tensor, sample_length_tensor)
             probabilities = F.softmax(logits, dim=-1)
             prediction_idx = torch.argmax(probabilities, dim=-1).item()
             confidence = probabilities[0][prediction_idx].item()

        predicted_sentiment = "positive" if prediction_idx == 1 else "negative"
        actual_sentiment = "positive" if sample_label == 1 else "negative"

        print("\n===== First Test Sample Prediction =====")
        print(f"Actual Sentiment: {actual_sentiment}")
        print(f"Predicted Sentiment: {predicted_sentiment}")
        print(f"Confidence: {confidence:.3f}")

        if hasattr(hparams, "id2word"):
            sample_text = ' '.join([hparams.id2word[idx] for idx in sample_text_ids if idx in hparams.id2word])
        else:
            sample_text = " ".join(map(str, sample_text_ids))

        print(f"Text: {sample_text}")

    # Free memory for later usage.
    del model
    torch.cuda.empty_cache()
    return {
           'num_params': num_params,
           "test_loss": epoch_test_loss,
           "test_acc": epoch_test_acc,
    }


### Lab 1 (f): Train LSTM model .

Train the model with default hyperparameter settings.

In [30]:
import pandas as pd

try:
    # Assuming you are reading a CSV file
    data = pd.read_csv('IMDBDataset.csv', on_bad_lines='skip')
except pd.errors.ParserError as e:
    print(f"ParserError: {e}")
    # Handle the error or log it as needed

In [31]:

org_hyperparams = HyperParams()
_ = train_and_test_model_with_hparams(org_hyperparams, "lstm_1layer_base_adam_e32_h100")


shape of train data is (35000,)
shape of test data is (10000,)
shape of valid data is (5000,)
Length of vocabulary is 37977
The model has 79,379 trainable parameters
evaluating...: 100%|██████████| 53/53 [00:01<00:00, 32.66it/s]

<ipython-input-29-e4165959ec14>:103: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_save_path))



epoch: 1
train_loss: 0.676, train_acc: 0.562
valid_loss: 0.645, valid_acc: 0.642
evaluating...: 100%|██████████| 105/105 [00:03<00:00, 32.34it/s]
test_loss: 0.643, test_acc: 0.644

===== First Test Sample Prediction =====
Actual Sentiment: negative
Predicted Sentiment: negative
Confidence: 0.530
Text: 1 1 1 689 1 11169 111 1 1 612 1 1 288 1 1 10449 1 111 1 1 9610 9264 110 1 707 1 6201 19952 110 1 1 1 4620 1 1 1 1 7067 1 15 1 462 1 1 1 111 10210 1 1 612 1 1 1 363 4052 1 14647 1 1 125 1 918 1 1936 478 1 3366 1 748 1 1 1346 1 1 1667 1 1 111 1 1122 1 1 55 1 818 1 1 1 271 1 1 1109 1 2073 1 1 18 1 1 1 1 25087 15 1 20 1 1 1 245 1 1 14 1 1 1 1 600 442 2702
evaluating...: 100%|██████████| 53/53 [00:01<00:00, 30.85it/s]
epoch: 2
train_loss: 0.420, train_acc: 0.816
valid_loss: 0.360, valid_acc: 0.862
evaluating...: 100%|██████████| 105/105 [00:03<00:00, 32.32it/s]
test_loss: 0.347, test_acc: 0.864

===== First Test Sample Prediction =====
Actual Sentiment: negative
Predicted Sentiment: negative
